In [9]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [11]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_97811/542325871.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_97811/542325871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_97811/542325871.py:5: SettingWithCo

# Creating dummies

In [12]:
# Creating dummies for each year
data_est = pd.get_dummies(data, columns=['Year'], drop_first=True)

# Creating dummy for china
data['China'] = (data['Country'] == 'CH').astype(int)

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_97811/389578803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['China'] = (data['Country'] == 'CH').astype(int)


In [13]:
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,China
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CH,257,1.0,1
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CH,183,1.0,1
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CH,177,1.0,1
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CH,3,1.0,1
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,0
29,3,2020,0.034680,Audi,e-tron,37.5,89.010141,40.2,17,SUV,F,DE,491,1.0,0
30,3,2021,0.010494,Audi,e-tron,37.5,80.003519,40.2,17,SUV,F,DE,260,1.0,0
31,3,2022,0.017570,Audi,e-tron,37.5,78.972366,40.2,17,SUV,F,DE,538,1.0,0
32,3,2023,0.001099,Audi,e-tron,37.5,67.303773,40.2,17,SUV,F,DE,68,1.0,0
41,4,2021,0.003391,Audi,e-tron GT,47.2,127.889611,52.2,17,Sedan,F,DE,84,1.0,0


# Creating log market share

In [14]:
# Take the log of the market share
data['log_market_share'] = np.log(data['Market share'])

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_97811/3167553905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['log_market_share'] = np.log(data['Market share'])


# Independent OLS

In [15]:
y = data['log_market_share']
x = data[['const', 'Range', 'Price', 'HP', 'Chargetime']]
dummies = data[['China']]
X = pd.concat([x, dummies], axis=1)
print(X.shape)

(334, 6)


In [16]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit(cov_type='HC3')
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     21.01
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           3.18e-18
Time:                        14:20:17   Log-Likelihood:                -716.37
No. Observations:                 334   AIC:                             1445.
Df Residuals:                     328   BIC:                             1468.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.4240      0.700     -9.171      0.0

In [17]:
coef = pd.DataFrame(OLS_result.params)
coef.shape

(6, 1)